In [1]:
import torch

In [2]:
import sys

sys.path.extend([
    "/home/fernando/git_research/vitsreliability/GroundingDINO",
    "/home/fernando/git_research/vitsreliability",
    "/home/fernando/git_research/libLogHelper/build",
])

## Test if the maximals is changing anything at the output

In [3]:
def count_errors(lhs: torch.Tensor, rhs: torch.Tensor, threshold=None) -> int:
    if threshold is not None:
        diff_tensor = torch.le(
            torch.abs(
                torch.subtract(lhs, rhs)
            ), threshold
        )
    else:
        diff_tensor = torch.eq(lhs, rhs)
    num_differences = torch.sum(diff_tensor == False)

    return num_differences.item()

In [4]:
gold_dino_base_unharded = torch.load("../data/groundingdino_swinb_cogcoor_fp32_hardenedid_grounding_dino_8_1.pt",
                                     map_location=torch.device('cpu'))
gold_dino_base_maximals = torch.load("../data/groundingdino_swinb_cogcoor_fp32_None_grounding_dino_8_1.pt",
                                     map_location=torch.device('cpu'))

In [5]:
golden_h, input_list_h, gt_targets_h, model_h, input_captions_h, text_encoder_type_h, coco_api_h, map_list_h = gold_dino_base_maximals
golden_u, input_list_u, gt_targets_u, model_u, input_captions_u, text_encoder_type_u, coco_api_u, map_list_u = gold_dino_base_unharded


In [6]:
for g_h, g_u in zip(golden_h, golden_u):
    for (k_gh, v_gh), (k_gu, v_gu) in zip(g_h.items(), g_u.items()):
        errors = count_errors(lhs=v_gh, rhs=v_gu)
        if errors != 0:
            print("Errors for", k_gh, " -- ", errors, "are keys equal", k_gh == k_gu)

## Testing if the geommetry parser works

In [18]:
import geometry_parser
import copy
import numpy

In [23]:
def get_highest_geometry(golden, output):
    geometry_type = geometry_parser.ErrorGeometry.MASKED
    threshold = 0.0
    for batch_out, batch_golden in zip(golden, output):
        # Compute the absolute difference
        diff = torch.abs(batch_out - batch_golden).numpy()
        # Create a boolean tensor where the difference exceeds the threshold
        diff_exceeds_threshold = diff > threshold
        print(numpy.sum(diff_exceeds_threshold))
        # Convert the boolean tensor to an integer tensor of zeros and ones
        result_matrix = diff_exceeds_threshold.astype(numpy.uint8)
        current_geometry = geometry_parser.geometry_comparison(diff=result_matrix)
        if geometry_type < current_geometry:
            print("passou", current_geometry)
            geometry_type = current_geometry
    return geometry_type

In [24]:
test_array_1 = torch.randn(4, 12, 12, 1024)
test_array_2 = copy.deepcopy(test_array_1)
test_array_2[2, 2, 3, 2] *= 2
print(get_highest_geometry(golden=test_array_1, output=test_array_2))

In [10]:
test_array_2[2, 2, 3].shape

In [11]:
test_array_2[2, 2].shape